<h1 align="center"> pyLDPC Tutorial: Images </h1>


## update:  02/25/16 - v.0.7.0

<b><font color="red"> Since version 0.7: Coding and decoding functions take tG (Transposed G) instead of G, the coding matrix. Functions that construct it (CodingMatrix and CodingMatrix_systematic) return tG instead of G as well. </font></b> 

<br>
<b><font color="blue"> New functions/sections are indicated in blue </font></b>



This notebook introduces a user's guide of pyLDPC's sub-module Images: ldpc_images.
If you would like to know what each function does and go into the construction details, go to <a href="http://nbviewer.jupyter.org/github/janatiH/pyldpc/blob/master/pyLDPC-Images-Construction.ipynb?flush_cache=true/"> LDPC-Images Construction Details</a>


<font color="blue"><h2> Outline:</h2></font> 
<font color="blue"><h2> --------------------------------------------------</h2></font> 

**I. Using small matrices: Pixel by pixel coding & decoding**

<font color="green"><i>(3 times faster compared to v0.6)</i></font>

      1.Grayscale Images.
      2.RGB Images. 
 
**II. Using large matrices: row by row coding & decoding**

      1. scipy.sparse CSR objects.
      2. Codign & decoding.
      3. About matrices construction.

<font color="blue"><i>(more efficient with low SNRs)</i></font>
      
**III. Conclusion **
<font color="blue"><h2> --------------------------------------------------</h2></font> 

We'll consider 2 types of images in this tutorial:

- Grayscale images: Images whean read by cv2.imread(), are seen as 2D arrays. Each pixel is a uint8 number (0-255). 
- RGB Images: Images that are seen as 3D arrays. Each pixel is an RGB array of 3 uint8 numbers ( [R,G,B] ). 


In [1]:
import numpy as np
from IPython.display import display, HTML
import pyldpc
from pyldpc import ldpc_images
import cv2 
from time import time


If I had to sum up Images Coding and decoding in a few words, I'd sat:

1. Read the image [2D-array if grayscale, 3D-array if RGB]. 
2. Make coding and decoding matrices G and H.
3. Binarize the image [Using functions we'll see later].
4. Apply Coding function to binary image.
5. Apply Decoding function to Coded image. 

We'll go step by step 1 through 5 for both grayscale and RGB Images.

# I - Pixel by pixel: small matrices

Picture the image as a huge array of uint8 numbers (meaning, **u** for unsigned (positive) **int** for integer and 8 for 8 binary bits, which means that the number is lower than 2^7). 

If the image is grayscale, each pixel equals one uint8 number. The image is therefore read as a 2D-array. 
If the image is colorful, each pixel equals a tuple of 3 uint8 numbers (one for each color, Red Green Blue). The image is therefore a 3D-array, the third dimension being equal to 3. 

Pixel by pixel coding is done by coding and decoding one pixel at a time:

- If the image is grayscale, we write each pixel as a 8-sized binary array and pass it to coding and decoding functions. 
- If the image is RGB, each pixel is written as a 24-sizes binary array as passed to coding and decoding functions.

So now you see:
- For **Gray** Images, the size of messages is 8, G must have 8 rows.
- For **RGB** Images, the size of messages is 24, G must have 24 rows.


## I -1 Grayscale: 

Let's code and decode the grayscale image: <img src="Images/eye.png"> 
Generate the matrices. <b>Make sure G has <u>8</u> rows for <u>grayscale</u> images.</b>*

* *Details about how to choose matrices parameters <a href="http://nbviewer.jupyter.org/github/janatiH/pyldpc/blob/master/pyLDPC-Tutorial-Matrices.ipynb?flush_cache=true/">here </a>, but I recommand going through this tutorial and using the matrices defined hereby for now, and then optimize your coding & decoding by changing the matrices.*

In [19]:
n = 24
d_c = 4
d_v = 3
H = pyldpc.RegularH(n,d_v,d_c)
H,tG= pyldpc.CodingMatrix_systematic(H)
tG.shape


(24, 8)

Load the image using cv2.imread:

<b><font color="red"> imread's second argument is important: 0 for grayscale images. If not mentioned, imread will read the image as an RGB image return a 3D-array which will cause errors in coding if a grayscale matrix is used! </font></b> 

In [8]:
eye = cv2.imread("Images/eye.png",0)

Transform image format from np.uint8 to binary: 

In [9]:
eye_bin = ldpc_images.Gray2Bin(eye)

Code and send binary image eye_bin through a channel with:

- SNR = 6 db


In [20]:
code = 1

if code:
    snr = 6
    eye_coded_6,eye_noisy_6 = ldpc_images.ImageCoding(tG,eye_bin,snr)

    ## Save the image locally: 
    cv2.imwrite("Images/eye/grayscale/snr=6/eye_noisy.jpg",eye_noisy_6)


### Let's have a look at  the noisy pictures:

### Channel AWGN snr = 6 db.


In [11]:
display(HTML('''<img src="Images/eye/grayscale/snr=6/eye_noisy.jpg">'''))


Let's compare the original picture to the noisy one: 

In [13]:
eye_compare_noisy_orginal_6 = np.concatenate((eye,eye_noisy_6),axis=1) 
cv2.imwrite("Images/eye/grayscale/snr=6/eye_compare_noisy_original.jpg",eye_compare_noisy_orginal_6)
display(HTML('''<img src="Images/eye/grayscale/snr=6/eye_compare_noisy_original.jpg">'''))


### Decoding (Full-log BP):
And now, let's decode:

<font color=#A44057> Warning ! Decoding takes time depending on the image's size and the matrices' size. (128 x 128) grayscale image (like this one) decoding may last for about 30 seconds! </font>



> Iteration number: 1

In [21]:
decode = 1

max_iter= 1
snr = 6
if decode:
    t = time()
    eye_decoded_6 = ldpc_images.ImageDecoding(tG,H,eye_coded_6,snr,max_iter)
    cv2.imwrite("Images/eye/grayscale/snr=6/iterations/eye_iter"+str(max_iter)+".jpg",eye_decoded_6)
    t = time() - t
    print("Decoding time for {} iterations and snr = {}: ".format(max_iter,snr),t) 
    display(HTML('''<img src="Images/eye/grayscale/snr=6/iterations/eye_iter'''+str(max_iter)+'''.jpg">'''))
print("Bit Error Rate in %:", 100*ldpc_images.BER(eye_bin,ldpc_images.Gray2Bin(eye_decoded_6)),"%")

Decoding time for 1 iterations and snr = 6:  23.89686679840088


Bit Error Rate in %: 0.102233886719 %


The decoding is not error-free, let's increment our decoding loop to 10 iterations: 

In [22]:
decode = 1

max_iter= 10
snr = 6
if decode:
    t = time()
    eye_decoded_6 = ldpc_images.ImageDecoding(tG,H,eye_coded_6,snr,max_iter)
    cv2.imwrite("Images/eye/grayscale/snr=6/iterations/eye_iter"+str(max_iter)+".jpg",eye_decoded_6)
    t = time() - t
    print("Decoding time for {} iterations and snr = {}: ".format(max_iter,snr),t) 
    display(HTML('''<img src="Images/eye/grayscale/snr=6/iterations/eye_iter'''+str(max_iter)+'''.jpg">'''))
print("Bit Error Rate in %:", 100*ldpc_images.BER(eye_bin,ldpc_images.Gray2Bin(eye_decoded_6)),"%")

Decoding time for 10 iterations and snr = 6:  27.30986499786377


Bit Error Rate in %: 0.0 %


Decoding is errror-free, let's compare the noisy image to the decoded one: (original on the left)

In [13]:
eye_compare_decoded_noisy_6 = np.concatenate((eye_noisy_6[:,:eye.shape[1]],eye_decoded_6),axis=1) 
cv2.imwrite("Images/eye/grayscale/snr=6/eye_compare_decoded_noisy_6.jpg",eye_compare_decoded_noisy_6)
display(HTML('''<img src="Images/eye/grayscale/snr=6/eye_compare_decoded_noisy_6.jpg">'''))


## I-2  RGB Images
<img src="Images/oeil.png"> 

Make sure Coding Matrix has 24 rows for RGB images*:

* *Details about matrices construction <a href="http://nbviewer.jupyter.org/github/janatiH/pyldpc/blob/master/pyLDPC-Tutorial-Matrices.ipynb?flush_cache=true/">here </a>*

In [34]:
n = 88
d_c = 4
d_v = 3
H = pyldpc.RegularH(n,d_v,d_c)
H,tG= pyldpc.CodingMatrix_systematic(H)
tG.shape

(88, 24)

In [24]:
rgbeye = cv2.imread("Images/oeil.png",1)
print("Shape of RGB image:",rgbeye.shape)
rgbeye_bin = ldpc_images.RGB2Bin(rgbeye)
print("Shape of Binary RGB image:",rgbeye_bin.shape)


Shape of RGB image: (128, 128, 3)
Shape of Binary RGB image: (128, 128, 24)


### Coding : SNR = 6

In [35]:
code = 1

if code:
    snr = 6
    rgbeye_coded_6,rgbeye_noisy_6 = ldpc_images.ImageCoding(tG,rgbeye_bin,snr)

    ## Save the image locally: 
    cv2.imwrite("Images/eye/rgb/snr=6/eye_noisy.jpg",rgbeye_noisy_6)


Noisy Image: 

In [17]:
display(HTML('''<img src="Images/eye/rgb/snr=6/eye_noisy.jpg">'''))

### Decoding 

Let's try one iteration of Full-log BP decoding:

<font color=#A44057> <b> Warning !</b> <br>
Decoding takes time depending on the image's size and number of iterations max_iter. (128 x 128) RGB image (like this one) decoding may last for 1 to 5 minutes depending on the size of the matrices and their sparsness properties ! </font> 




In [36]:
decode = 1

max_iter= 1
snr = 6
if decode:
    t = time()
    rgbeye_decoded_6 = ldpc_images.ImageDecoding(tG,H,rgbeye_coded_6,snr,max_iter)
    cv2.imwrite("Images/eye/rgb/snr=6/iterations/eye_iter"+str(max_iter)+".jpg",rgbeye_decoded_6)
    t = time() - t
    print("Decoding time for {} iterations and snr = {}: ".format(max_iter,snr),t) 
    display(HTML('''<img src="Images/eye/rgb/snr=6/iterations/eye_iter'''+str(max_iter)+'''.jpg">'''))
print("Bit Error Rate in %:", 100*ldpc_images.BER(rgbeye_bin,ldpc_images.RGB2Bin(rgbeye_decoded_6)),"%")

Decoding time for 1 iterations and snr = 6:  92.55027103424072


Bit Error Rate in %: 0.0762939453125 %


One iteration is not enough, but you can increase max_iter to get better decoding.

In [52]:
rgbeye_compare_decoded_noisy_6 = np.concatenate((rgbeye_noisy_6[:,:rgbeye.shape[1]],rgbeye_decoded_6),axis=1) 
cv2.imwrite("Images/eye/rgb/snr=6/rgbeye_compare_decoded_noisy_6.jpg",rgbeye_compare_decoded_noisy_6)
display(HTML('''<img src="Images/eye/rgb/snr=6/rgbeye_compare_decoded_noisy_6.jpg">'''))


# II - Row by row: Large matrices:

Imagine you transform each row of an image to a very long binary array. And then code and decode the image row by row. This can't be done (in practice at least) with numpy arrays:
- Coding matrix construction takes **A LOT** of time.
- Coding and decoding has a quadratic complexity, which will also take **A LOT** of time. 

To speed calculations with large matrices, <font color="blue"> pyLDPC introduced in v0.7 the use of *scipy.sparse.csr_format*. </font> I made sure to guaratee a sort of backward compatibility with numpy arrays, so wether you use numpy arrays or CSR objects, it doesn't matter: pyLDPC *understands* both. 

## II - 1 CSR objects: 
You can create a CSR object of a decoding matrix H by:

```python
from scipy.sparse import csr_matrix
Hs = csr_matrix(H)
# You can get the array form back by computing:
Harray = Hs.toarray()
```
That's it. Hs is sort of compressed format *(CSR: compressed storage rows)* where only keys and values of nonzeros elements are stored. This is very efficient when dealing with sparse matrices like parity-check matrices. CSR objects allow faster matrix-vector products, faster matrix-matrix products (which is used in CodingMatrix and CodingMatrix_systematic functions if n is very high). 

## II - 2 Coding and decoding:

### How to construct H and G ?

* *This section only covers a practical example of H,G construction, theoritical intuition and details about matrices construction <a href="http://nbviewer.jupyter.org/github/janatiH/pyldpc/blob/master/pyLDPC-Tutorial-Matrices.ipynb?flush_cache=true/">here </a>*

To construct or <font color="blue"> load from the library </font> H and tG, you need to evaluate k, the number of bits per row in your image. 
In this example: 

<img src="Images/eye.png">

The grayscale image has a shape 128x128. Since each pixel is an 8 bits array, the image has 128x8 = 1024 bits per row.

If I want a code of rate k/n approximately equal to 1/3, I'll have to

1- fix dv and dc so that ** 1 - dv/dc = 1/3.** Knowing that the quality of decoding is better the less H has nonzeros elements, dv and dc must be as low as possible: **dc = 3, dv=2. **
2- try n = 3k, n = 3k - dc, n = 3k - 2dc .. until you find k = 1024. 

This trick, as lame as it looks like, is not that bad: 

- For Rate = 1/3: apply n = 3k - 3 
- For Rate = 1/4: apply n = 4k - 8 

It actually works! 

In [39]:
d_c,d_v = 3,2
n = 1024*3 - 3 
H = pyldpc.RegularH(n,d_v,d_c)
H,tG = pyldpc.CodingMatrix_systematic(H)
tG.shape

(3069, 1024)

Indeed ! k = 1024. Now, put matrices in CSR format: 

In [40]:
from scipy.sparse import csr_matrix
Hs, tGs = csr_matrix(H),csr_matrix(tG)

Let's code the grayscale image using the <font color="blue"> new function ImageCoding_rowbyrow </font>.

### Coding:

In [42]:
code = 1

if code:
    snr = 6
    eye_coded_6r,eye_noisy_6r = ldpc_images.ImageCoding_rowbyrow(tGs,eye_bin,snr)

    ## Save the image locally: 
    cv2.imwrite("Images/eye/grayscale/snr=6/eye_noisyr.jpg",eye_noisy_6r)


In [43]:
display(HTML('''<img src="Images/eye/grayscale/snr=6/eye_noisyr.jpg">'''))


### Decoding full-log BP: 


In [44]:
decode = 1

max_iter= 1
snr = 6
if decode:
    t = time()
    eye_decoded_6r = ldpc_images.ImageDecoding_rowbyrow(tGs,Hs,eye_coded_6r,snr,max_iter)
    cv2.imwrite("Images/eye/grayscale/snr=6/iterations/eye_iter"+str(max_iter)+".jpg",eye_decoded_6r)
    t = time() - t
    print("Decoding time for {} iterations and snr = {}: ".format(max_iter,snr),t) 
    display(HTML('''<img src="Images/eye/grayscale/snr=6/iterations/eye_iter'''+str(max_iter)+'''.jpg">'''))
print("Bit Error Rate in %:", 100*ldpc_images.BER(eye_bin,ldpc_images.Gray2Bin(eye_decoded_6r)),"%")

Decoding time for 1 iterations and snr = 6:  22.968536853790283


Bit Error Rate in %: 0.0823974609375 %


Let's increase the number of iterations:

In [45]:
decode = 1

max_iter= 10
snr = 6
if decode:
    t = time()
    eye_decoded_6r = ldpc_images.ImageDecoding_rowbyrow(tGs,Hs,eye_coded_6r,snr,max_iter)
    cv2.imwrite("Images/eye/grayscale/snr=6/iterations/eye_iter"+str(max_iter)+".jpg",eye_decoded_6r)
    t = time() - t
    print("Decoding time for {} iterations and snr = {}: ".format(max_iter,snr),t) 
    display(HTML('''<img src="Images/eye/grayscale/snr=6/iterations/eye_iter'''+str(max_iter)+'''.jpg">'''))
print("Bit Error Rate in %:", 100*ldpc_images.BER(eye_bin,ldpc_images.Gray2Bin(eye_decoded_6r)),"%")

Decoding time for 10 iterations and snr = 6:  44.67349100112915


Bit Error Rate in %: 0.0 %


Coding RGB images row by row is very similar. Only the number of bits per row changes. Instead of computing k = 8 * pixels/row , compute k = 24*pixels/row. 

## II - 3 About Matrices construction: 

When dealing with RGB images, you may have to construct matrices with thousands of rows and columns. That's why, I created a sort of library where I stored some ldpc matrices.

- <a href="http://nbviewer.jupyter.org/github/janatiH/pyldpc/blob/master/pyLDPC-Tutorial-Matrices.ipynb?flush_cache=true/"> Matrices Construction details </a>.

- <font color="red">work in progress </font> <a href="http://nbviewer.jupyter.org/github/janatiH/pyldpc/blob/master/pyLDPC-Library-Matrices.ipynb?flush_cache=true/"> Matrices Library Access Tutorial </a>.

Before constructing any matrices, you should probably check if it's not already created in the library. 